In [11]:
import pandas as pd
import numpy as np
import time
import datetime
# install plotly thru conda if jupyter cannot locate the module thru pip
import plotly.express as px
import plotly.graph_objects as go #object properties (figure)
from ipywidgets import interact
from tabulate import tabulate

# loading data from the source:

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

# helper functions

def norm(val):
    val = (val - val.mean()) / val.std()
    return val

def format_int(val):
    val = '{:,}'.format(val)
    return val

def highlight_col(x):
    r = 'background-color: red'
    a = 'background-color: yellow'
    g = 'background-color: green'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = a
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1

    
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})


# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

table = [["Stats as of", "Confirmed total", "Deaths total", "Recovered total"],
         [st, format_int(confirmed_total), format_int(deaths_total), format_int(recovered_total)]]

print(tabulate(table, headers="firstrow", tablefmt="github", numalign="center", stralign="center"))

#print("Stats as of " + st)
#print("Confirmed total: ", format_int(confirmed_total))
#print("Deaths total: ", format_int(deaths_total))
#print("Recovered total: ", format_int(recovered_total))

sorted_country_df = country_df.sort_values('confirmed', ascending= False)

# vizualizations
def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

def show_by_country(country):
    country = str(country)
    return country_df[country_df['country'].str.contains(country)].style.apply(highlight_col, axis=None)

def worst_hit_bar(num):
    fig = px.bar(sorted_country_df.head(num),
    x = "country",
    y = "deaths")
    fig.update_layout(
    title=str(num)+" Most affected countries",
    height=500,
    width=800
    )
    fig.show()

def worst_hit_scatter(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Most Affected Countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show()
    
def cases_per_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country.lower() == 'global'.lower():
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19-cases: " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();    
    
interact(worst_hit_scatter, n=10)
interact(worst_hit_bar, num=5)
interact(cases_per_country, country='US')
interact(show_latest_cases, n='10')
interact(show_by_country, country='US')


|     Stats as of     |  Confirmed total  |  Deaths total  |  Recovered total  |
|---------------------|-------------------|----------------|-------------------|
| 2020-07-10 01:10:54 |    12,268,518     |    554,924     |     6,740,124     |


interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=5, description='num', max=15, min=-5), Output()), _dom_classes=('widget-…

interactive(children=(Text(value='US', description='country'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='US', description='country'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_by_country(country)>